This will will introduce you to Logs, Traces, and Metrics in Snowflake which can be used to optimize performance and find errors in UDFs and stored procedures.

You will use REST APIs from OpenWeather to showcase the fetch weather data as part of a pipeline.

I would have normally done this in our overall setup script, but I wanted to show how we're setting our tracing configuration.

Make sure you add the opentelemetry-api and opentelemtry-distro packages via the 'packages' menu in the top right

In [ ]:
CREATE OR REPLACE WAREHOUSE TRACING_QUICKSTART_WH WAREHOUSE_SIZE=XSMALL, INITIALLY_SUSPENDED=TRUE;
CREATE OR REPLACE DATABASE TRACING_QUICKSTART;
CREATE OR REPLACE SCHEMA DATA;

ALTER DATABASE TRACING_QUICKSTART SET TRACE_LEVEL = ALWAYS;
ALTER SCHEMA DATA SET TRACE_LEVEL = ALWAYS;

### Parameters for event tables
You can use the following parameters to specify how the event table should be used by handler code.

**EVENT_TABLE**

Specifies the name of the event table for logging messages from stored procedures and UDFs in this account. For reference information, see EVENT_TABLE.

**LOG_LEVEL**

Specifies the severity level of messages that should be ingested and made available in the active event table. Messages at the specified level (and at more severe levels) are ingested. For more information, see LOG_LEVEL and Setting levels for logging, metrics, and tracing.

**METRIC_LEVEL**

Specifies whether metrics data should be ingested and made available in the active event table. For more information, see METRIC_LEVEL and Setting levels for logging, metrics, and tracing.

**TRACE_LEVEL**

Specifies the verbosity of trace events that should be ingested and made available in the active event table. Events at the specified level are ingested. For more information, see TRACE_LEVEL and Setting levels for logging, metrics, and tracing.

### TRACE_LEVEL
**Type**

Session — Can be set for Account » User » Session

Object (for databases, schemas, stored procedures, and UDFs) — Can be set for Account » Database » Schema » Procedure and Account » Database » Schema » Function

**Data Type** String (Constant)

**Description**

Controls how trace events are ingested into the event table. For more information about trace levels, see Setting levels for logging, metrics, and tracing.

**Values**

ALWAYS: All spans and trace events will be recorded in the event table.

ON_EVENT: Trace events will be recorded in the event table only when your stored procedures or UDFs explicitly add events.

OFF: No spans or trace events will be recorded in the event table.

**Default** OFF

**Note**

When tracing events, you must also set the LOG_LEVEL parameter to one of its supported values. (We will do this later)

Now, we are going to use OpenWeather API data. You will need to [sign up](https://home.openweathermap.org/users/sign_up) for an account on OpenWeather.

After signing up, add a new API key by going to API keys. [Find your API key](https://home.openweathermap.org/api_keys) and copy the API key to the clipboard.

These variables will be used in subsequent cells using the {{variable_name}} syntax in SQL cells, and by using the variable names in Python cells.

When running this cell, it will prompt for entering an API key. This was done with the streamlit text_input widget.

To get a free API key, sign up for an account on OpenWeather.

After signing up, add a new API key by going to API keys. Create a new key with name snowflake_key and click Generate, copy the API key to the clipboard.

After generating the key, go back to the Snowsight notebook and paste the key generated in the dialog and hit enter.

In [ ]:
# These variables will be used in subsequent cells using the {{variable_name}} syntax in SQL cells, and by using the variable names in Python cells.

import streamlit as st

from snowflake.snowpark.context import get_active_session
session = get_active_session()

user_name = session.sql("SELECT current_user()").collect()[0][0]
schema_name = "DATA"
database_name = "TRACING_QUICKSTART"

city_list = [(37.5533204, -122.3059259, 'San Mateo'),(43.6532, -79.3832, 'Toronto'), (52.3368551, 4.8694973, 'Amsterdam'),(52.5100227, 13.3776724, 'Berlin'), (52.2299305,20.9764142, 'Warsaw'), (18.5645333,73.911966, 'Pune')]

api_key = "<REPLACE>"

Here you will configure a network rule, secret, and external access integration to allow outbound connectivity to the OpenWeather API. This is necessary because the Python Notebook is secure by default and does not allow network access. A stage is also created to store the permanent UDF and procedure. "Permanent" means that the UDF and stored procedure will be stored in the database so that they can be called from outside the Notebook in the future, such as from a SQL query.

In [ ]:
USE DATABASE "{{database_name}}";
USE SCHEMA "{{schema_name}}";

CREATE OR REPLACE NETWORK RULE OPENWEATHERMAP_API_NETWORK_RULE
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('api.openweathermap.org');

CREATE OR REPLACE SECRET OPENWEATHERMAP_API_KEY
    TYPE = GENERIC_STRING
    SECRET_STRING = "{{api_key}}";

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION OPENWEATHERMAP_ACCESS_INTEGRATION
  ALLOWED_NETWORK_RULES = (OPENWEATHERMAP_API_NETWORK_RULE)
  ALLOWED_AUTHENTICATION_SECRETS = (OPENWEATHERMAP_API_KEY)
  ENABLED = true;

CREATE STAGE IF NOT EXISTS EXEC_STORAGE;

In order to get logs and metrics, levels need to be modified. This can easily be done in SQL.

**LOG_LEVEL** Specifies the severity level of messages that should be ingested and made available in the active event table. Messages at the specified level (and at more severe levels) are ingested.

**METRIC_LEVEL** Controls how metrics data is ingested into the event table. For more information about metric levels, 

In [ ]:
ALTER DATABASE "{{database_name}}" SET LOG_LEVEL = DEBUG;
ALTER SCHEMA "{{schema_name}}" SET LOG_LEVEL = DEBUG;

ALTER DATABASE "{{database_name}}" SET METRIC_LEVEL = ALL;
ALTER SCHEMA "{{schema_name}}" SET METRIC_LEVEL = ALL;

ALTER SESSION SET METRIC_LEVEL = ALL;

A UDF will use the External Access and Secret created previously and pull data from the OpenWeather API using Python Requests library. It will pull the current weather for a specific latitude and longitude.

In order to populate the bronze layer, a stored procedure will call the UDF for every city in the city_list to get the current weather. This is done with a dataframe and will append the data to a table for usage in the future.

When calling to a 3rd party API like this, it is important to know the performance and potential errors coming from those calls. To trace this information, the function uses a custom span in OpenTelemetry. Review the tracer variable and how that is used. Without the custom span, it would be difficult to know which calls were erroring out to the API and what status code the API was returning.

The function will be stored in the database schema and can be used in the future from SQL or Python, as we will do in the next cell.

In [ ]:
import _snowflake
import requests
import json
import logging


rsession = requests.Session()
def get_weather(lat, lon):
  """
  A UDF handler. Given a latitude and longitude, this UDF will use external network access to get the 
  weather for that area from the OpenWeatherMap API.
  """
  api_key = _snowflake.get_generic_secret_string('api_key')
  url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&exclude=hourly,daily&appid={api_key}"
  response = rsession.get(url)
  logging.debug(f"Body from API: {response.text} in get_weather")
  if response.status_code != 200:
    logging.warn(f"Unexpected response from API: {response.status_code} in get_weather")
  return response.json()

# Register the UDF
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import VariantType, FloatType, IntegerType, StringType
session = get_active_session()

get_weather_fn = session.udf.register(get_weather,
                                   return_type=VariantType(),
                                   input_types=[FloatType(),FloatType()],
                                   name="get_weather_fn",
                                   replace=True,
                                   is_permanent=True,
                                   stage_location="EXEC_STORAGE",
                                   secrets={'api_key':'OPENWEATHERMAP_API_KEY'},
                                   external_access_integrations=["OPENWEATHERMAP_ACCESS_INTEGRATION"],
                                   packages=["snowflake-snowpark-python", "requests", "snowflake-telemetry-python"])

In [ ]:
-- Here we going to make a call and see if it works.
SELECT GET_WEATHER_FN(43.6532, -79.3832)

Now we can go ahead and pull the weather for all the cities in our city_list

In [ ]:
import datetime 
import time
import snowflake.snowpark

from opentelemetry import trace
from snowflake.snowpark.functions import sproc


def get_weather_for_cities(session, to_table, minutes_to_run, seconds_to_wait):
    """
    A stored procedure handler that reads from the city_list table and calls the UDF to add the wather
    data as a new column
    """
    df = session.table("city_list")
    stop_time = datetime.datetime.utcnow() + datetime.timedelta(minutes = minutes_to_run)
    tracer = trace.get_tracer(__name__)
    while datetime.datetime.utcnow() < stop_time:
        with tracer.start_as_current_span(f"get_weather_for_all_cities") as p:
            pdf = df.select(get_weather_fn("lat", "lon").alias("current_weather"), "name")
            pdf.write.mode("append").save_as_table(to_table)
        time.sleep(seconds_to_wait)
    return "OK"

# Store the city_list for use in the procedure
df = session.create_dataframe(city_list).to_df("lat","lon","name")
df.write.mode("overwrite").save_as_table("city_list")

# Register the stored procedure
get_weather_for_cities_sp = session.sproc.register(get_weather_for_cities, 
                                                   name="get_weather_for_cities_sp",
                                                   return_type=StringType(),
                                                   input_types=[StringType(),IntegerType(),IntegerType()],
                                                   is_permanent=True,
                                                   replace=True,
                                                   packages=["snowflake-snowpark-python", "requests", "snowflake-telemetry-python", "opentelemetry-api"],
                                                   stage_location="EXEC_STORAGE")

We are introducing a delay in this stored procedure, it will run, then sleep for 15 seconds, then run again, until it's ran for 1 minute.

After running, the notebook will output the dataframe of bronze_weather_api which includes the current weather for all the cities in city_list.

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql("call get_weather_for_cities_sp('bronze_weather_api', 1, 15)").collect()
session.table("bronze_weather_api")

And we can query the table that the stored procedure created via sql below

In [ ]:
SELECT NAME, 
       CURRENT_WEATHER['main']['temp']::float as KELVIN_TEMP, 
       CURRENT_WEATHER['weather'][0]['main']::varchar as CONDITIONS 
FROM bronze_weather_api
QUALIFY ROW_NUMBER() over (partition by NAME order by CURRENT_WEATHER['dt'] desc) = 1;


Now we are going to view the traces. We can do this via sql or we can do it via Snowsight.

We'll start with viewing the data via sql.

Memory and CPU metrics are also available because the session set the METRIC_LEVEL to all.

To get to this data, it is in the Events table.

In [ ]:
SHOW PARAMETERS LIKE 'event_table' IN ACCOUNT;

In [ ]:
-- Query the memory and cpu metrics with this query, replacing YOUR_EVENT_TABLE with your event table found in the previous query:

SELECT *
FROM SNOWFLAKE_TRAIL.TELEMENTRY.events
WHERE
    RECORD_TYPE = 'SPAN' 
    and RESOURCE_ATTRIBUTES['db.user'] = '{{user_name}}' 
    and RESOURCE_ATTRIBUTES['snow.database.name'] = '{{database_name}}' 
    and RESOURCE_ATTRIBUTES['snow.schema.name'] = '{{schema_name}}'
ORDER BY TIMESTAMP DESC;

In [ ]:
-- and we query the logs in the same way
SELECT *
FROM SNOWFLAKE_TRAIL.TELEMENTRY.events
WHERE
    RECORD_TYPE = 'LOG' 
    and RESOURCE_ATTRIBUTES['db.user'] = '{{user_name}}' 
    and RESOURCE_ATTRIBUTES['snow.database.name'] = '{{database_name}}' 
    and RESOURCE_ATTRIBUTES['snow.schema.name'] = '{{schema_name}}'
ORDER BY TIMESTAMP DESC;

To view this data in Snowsight, Click on the left navigation item "Monitoring" > "Traces & Logs".

Traces can take a minute to show up, so you may have to wait and refresh until they appear. You may have to filter using the Database filter and selecting TRACING_QUICKSTART if there are other traces in the account.

Look for the trace with the name GET_WEATHER_FOR_CITIES_SP and open to see the span details. Notice the get_weather_for_all_cities spans which are the more time consuming parts of the procedure. You can see fetching the data from the API and saving is only a few seconds, but the entire runtime of the procedure is over a minute.

If you expand the get_weather_fn entry under get_weather_for_all_cities, you can see the Debug Logs by clicking on Logs on the trace to see all the http request information.

This tracing information shows the entire execution timeline with information on every call.

To clean up, we'll go ahead and change the tracing, metrics and logging level similar to how we enabled it.

In [ ]:
ALTER DATABASE "{{database_name}}" SET LOG_LEVEL = WARN;
ALTER SCHEMA "{{schema_name}}" SET LOG_LEVEL = WARN;

ALTER DATABASE "{{database_name}}" SET TRACE_LEVEL = OFF;
ALTER SCHEMA "{{schema_name}}" SET TRACE_LEVEL = OFF;

ALTER DATABASE "{{database_name}}" SET METRIC_LEVEL = NONE;
ALTER SCHEMA "{{schema_name}}" SET METRIC_LEVEL = NONE;

ALTER SESSION SET METRIC_LEVEL = NONE;